In [9]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [10]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')


OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

!ollama pull llama3.2
    

OpenAI API Key exists and begins sk-proj-
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [11]:
openai = OpenAI()
system_message = "You are a helpful assistant"

def user_prompt(input_message):
    user_prompt = f'{input_message}'
    return user_prompt

In [76]:
import requests
import json

def get_chat_prompt(text):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt(text)}
    ]
    
def getllama(text):
    payload = {
        "model": MODEL,
        "messages": get_chat_prompt(text),
        "stream": True
    }
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True)
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line)
                content = data.get('message', {}).get('content', '')
                if content:
                    yield content
            except json.JSONDecodeError:
                continue
                
def getchat_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [77]:
getllama('what is python')

<generator object getllama at 0x128d4f540>

In [84]:
def stream_model(prompt, model):
    accumulated_text = ""
    
    if model == "llama":
        stream_gen = getllama(prompt)
    elif model == "Chat_gpt":
        stream_gen = getchat_gpt(prompt)
    else:
        raise ValueError(f"Unknown model: {model}")
    
    for text_chunk in stream_gen:
        accumulated_text += text_chunk
        yield accumulated_text 

In [86]:
view = gr.Interface(
    fn=stream_model,
    inputs=[
        gr.Textbox(label="Your message:", lines=3),
        gr.Dropdown(["llama", "Chat_gpt"], label="Select model", value="Chat_gpt")
    ],
    outputs=gr.Textbox(label="Response:", lines=10),
    live=False,  # Don't auto-run while typing
    flagging_mode="never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7894

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/al/Desktop/LLM Engineering- Master AI, LLM/projects/llm_engineering/venv/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/al/Desktop/LLM Engineering- Master AI, LLM/projects/llm_engineering/venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/al/Desktop/LLM Engineering- Master AI, LLM/projects/llm_engineering/venv/lib/python3.12/site-packages/gradio/blocks.py", line 2146, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/al/Desktop/LLM Engineering- Master AI, LLM/projects/llm_engineering/venv/lib/python3.12/site-pa